In [1]:
# coding: utf-8
import os
import xlrd
import re
import csv
import fasttext
import traceback
import random
import utils
import json
import jieba
import pandas as pd
from utils import fixText

def getConfig():
    config = {}
    with open('./config.json', 'r') as f:
        s = f.read()
        config = json.loads(s)
    return config

In [ ]:
with open('data/train.tsv', 'r') as f:
    content = f.readlines()
    random.shuffle(content)
    train = content[0:len(content) // 5 * 4]
    test = content[len(content) // 5 * 4:]
    
    with open('data/old_train_set.txt', 'w') as out:
        for c in train:
            res = re.search(r'__label__(.*)[^\r\n]', c)
            label = c[res.span()[0]:res.span()[1]]
            c = re.sub(r'__label__(.*)', '', c)
            c = ''.join(re.split('\t| |\r|\n', c))
            c = fixText(c)
            out.write(c + ' ' + label + '\n')
    with open('data/old_test_set.txt', 'w') as out:
        for c in test:
            res = re.search(r'__label__(.*)[^\r\n]', c)
            label = c[res.span()[0]:res.span()[1]]
            c = re.sub(r'__label__(.*)', '', c)
            c = ''.join(re.split('\t| |\r|\n', c))
            c = fixText(c)
            # c = ''.join(c.split(' '))
            out.write(c + ' ' + label + '\n')
        

In [ ]:
config = getConfig()
model = fasttext.train_supervised(
    input = './data/old_train_set.txt',
    lr = config['lr'],
    dim = config['hidden_dim'],
    epoch = config['epoch']
)

In [ ]:
total = 0
true_tag = 0
with open('./data/old_test_set.txt', 'r') as f:
    content = f.readlines()
    total = len(content)
    for c in content:
        res = re.search(r'__label__(.*)[^\r\n]', c)
        label = c[res.span()[0]:res.span()[1]]
        c = re.sub(r'__label__(.*)', '', c)
        predict = model.predict(c[:-1])[0][0]
        
        print(model.predict(c[:-1]))
        # print(predict, label)
        if(predict == label):
            true_tag += 1
        break
#print('tag level 1 accurate: {}% ({}/{})'.format(true_tag * 100 / total, true_tag, total))

In [24]:
import re
def resetLabelLv2(label):
    if(re.search(r'.+街(道)*', label)):
        label = re.search(r'.+街(道)*', label).group()
        if "街道" not in label:
            label = label.replace("街", "街道")
        if(re.search(r'.+[区|市]', label)):
            label = re.sub(re.search(r'.+[区|市]', label).group(), '', label)
    return label

def resetLabelLv3(label):
    if(re.search(r'[\u4e00-\u9fa50-9]+街(道)*', label)):
        label = re.search(r'街(道)*.*', label).group()
        label = re.sub(r'[(|（][\u4e00-\u9fa50-9]*[)|）]', '', label)
    return label


In [ ]:
addr = '广州市白云区区鹤洞街办事处假装有这个分队(测试一下括号内）测试一下括号外（测试一下多个括号）'
addr2 = resetLabelLv2(addr)
addr3 = resetLabelLv3(addr)
print(addr2)
print(addr3)


In [1]:
import json
from utils import resetLabelLv2, resetLabelLv3
with open('./data/train_set.json', 'r') as f:
    s = f.read()
    data_set = json.loads(s)
label1 = {}
label2 = {}
label3 = {}

for id in list(data_set.keys()):
    if data_set[id]['tag_level_1'] in label1:
        label1[data_set[id]['tag_level_1']] += 1
    else:
        label1[data_set[id]['tag_level_1']] = 1

    tag = resetLabelLv2(data_set[id]['tag_level_2'])
    if tag in label2:
        label2[tag] += 1
    else:
        label2[tag] = 1

    tag = resetLabelLv3(data_set[id]['tag_level_3'])
    if tag in label3:
        label3[tag] += 1
    else:
        label3[tag] = 1

In [ ]:
print(len(label1))
print(label1)

In [4]:
import fasttext
model = fasttext.FastText.load_model('./model/model_for_label_2.model')
len(model.words)

146222

In [3]:
import re
def split_text(text):
    splited_text = re.split(r'\n+|\r+|\r\n|<br>', text)
    while '' in splited_text:
        splited_text.remove('')
    return splited_text


In [8]:
s = "abc\n\ndef<br>\n\r\nghi"
print(re.split(r'\n+|\r+|\r\n|<br>', s))
print(split_text(s))

['abc', 'def', '', '', '', 'ghi']
['abc', 'def', 'ghi']


# 旧数据处理，划分训练和测试集

In [54]:
import random
import re
from utils import fixText
data_path = '../dispatch/train_data/train.tsv'
all_data = []
with open(data_path, 'r') as f:
    all_data = f.readlines()
    random.shuffle(all_data)
train_data = all_data[:len(all_data) // 5 * 4]
with open('../dispatch/train_data/new_train.tsv', 'w') as f:
    for item in train_data:
        f.write(item)
with open('./data/old_train_set.txt', 'w') as f:
    for item in train_data:
        res = re.search(r'__label__(.*)[^\r\n]', item)
        label = item[res.span()[0]:res.span()[1]]
        text = item[:res.span()[0]]
        text = re.sub(r'[ |\t]', '', text)
        text = fixText(text)
        f.write(label + ' ' + text + '\n')

test_data = all_data[len(all_data) // 5 * 4:]
with open('../dispatch/train_data/new_test.tsv', 'w') as f:
    for item in test_data:
        f.write(item)
with open('./data/old_test_set.txt', 'w') as f:
    for item in test_data:
        res = re.search(r'__label__(.*)[^\r\n]', item)
        label = item[res.span()[0]:res.span()[1]]
        text = item[:res.span()[0]]
        text = re.sub(r'[ |\t]', '', text)
        text = fixText(text)
        f.write(label + ' ' + text + '\n')

## 上家数据集测试

In [64]:
import fasttext
old_model = fasttext.load_model('../dispatch/models/model.bin')

In [79]:
import re
true_cnt = 0
total_num = 0
with open('../dispatch/train_data/new_test.tsv', 'r') as f:
    test_data = f.readlines()
for item in test_data:
    res = re.search(r'__label__(.*)[^\r\n]', item)
    label = item[res.span()[0]:res.span()[1]]
    text = re.sub(r'__label__(.*)[\n]', '', item)
    pre = old_model.predict(text)[0][0]
    if label == pre:
        true_cnt += 1
    total_num += 1

print('{} % ({}/{})'.format(true_cnt / total_num * 100, true_cnt, total_num))

83.24553366926249 % (14538/17464)


## 我们的模型应用到上家数据测试

In [55]:
from model import MyFastText

model = MyFastText()
model.train('./data/old_train_set.txt', lr = 0.1, dim = 256, epoch=30)

Read 4M words
Number of words:  96746
Number of labels: 43
Progress: 100.0% words/sec/thread:   22507 lr:  0.000000 avg.loss:  0.349475 ETA:   0h 0m 0s


In [80]:
import re
true_cnt = 0
total_num = 0
test_data = []
with open('./data/old_test_set.txt', 'r') as f:
    test_data = f.readlines()
for item in test_data:
    item = item.split(' ')
    label = item[0]
    text = ' '.join(item[1:])
    pre = model.predict(text)[0][0]
    if label == pre:
        true_cnt += 1
    total_num += 1

print('{} % ({}/{})'.format(true_cnt / total_num * 100, true_cnt, total_num))

83.77233165368759 % (14630/17464)


## 上家应用到新的数据集

In [2]:
from utils import resetLabelLv2, resetLabelLv3
def readDataSet(config):
    set1 = []
    with open(config['data_path'] + config['dataset_label_1_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            text = data_set[id]['text']
            # splited_text = split_text(text)
            # for text in splited_text:
            text = ''.join(fixText(text).split(' '))
            set1.append(' '.join(list(text)) + ' ' + '__label__' + data_set[id]['label_level_1'])
    
    set2 = []
    with open(config['data_path'] + config['dataset_label_2_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            text = data_set[id]['text']
            # splited_text = split_text(text)
            # for text in splited_text:
            text = ''.join(fixText(text).split(' '))
            set2.append(' '.join(list(text)) + ' ' + '__label__' + data_set[id]['label_level_2'])
    
    set3 = []
    with open(config['data_path'] + config['dataset_label_3_name'], 'r') as f:
        s = f.read()
        data_set = json.loads(s)
        for id in list(data_set.keys()):
            text = data_set[id]['text']
            # splited_text = split_text(text)
            # for text in splited_text:
            text = ''.join(fixText(text).split(' '))
            set3.append(' '.join(list(text)) + ' ' + '__label__' + data_set[id]['label_level_3'])
        
    with open('../dispatch/train_data/set1.txt', 'w') as f:
        print('训练集1拆分后共{}条文本'.format(len(set1)))
        for l in set1:
            f.write(l + '\n')
    with open('../dispatch/train_data/set2.txt', 'w') as f:
        print('训练集2拆分后共{}条文本'.format(len(set2)))
        for l in set2:
            f.write(l + '\n')
    with open('../dispatch/train_data/set3.txt', 'w') as f:
        print('训练集3拆分后共{}条文本'.format(len(set2)))
        for l in set3:
            f.write(l + '\n')

In [3]:
import json
config = getConfig()
readDataSet(config)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.726 seconds.
Prefix dict has been built successfully.


训练集1拆分后共153320条文本
训练集2拆分后共153320条文本
训练集3拆分后共153320条文本


In [7]:
from model import MyFastText
import fasttext
model_label1 = fasttext.load_model('../dispatch/models/model_for_set1.bin')
model_label2 = fasttext.load_model('../dispatch/models/model_for_set2.bin')
model_label3 = fasttext.load_model('../dispatch/models/model_for_set3.bin')

In [10]:
from utils import fixText
config = getConfig()
with open(config['data_path'] + 'test_set.json', 'r') as f:
    content = json.loads(f.read())

total = 0
true_lable_1 = 0
true_lable_2 = 0
true_lable_3 = 0

not_liwan_correct = 0
not_liwan_total = 0

for id in content.keys():
    text = content[id]['text']
    
    label_1 = content[id]['label_level_1']
    label_2 = content[id]['label_level_2']
    label_3 = content[id]['label_level_3']
    
    
    predict_1 = model_label1.predict(fixtext(text))[0][0]
    predict_2 = model_label2.predict(text[:-1])[0][0]
    predict_3 = model_label3.predict(text[:-1])[0][0]

    if(predict_1.replace('__label__', '') == label_1):
        true_lable_1 += 1
    if(predict_2.replace('__label__', '') == label_2):
        true_lable_2 += 1
    if(predict_3.replace('__label__', '') == label_3):
        true_lable_3 += 1
    total += 1

    if('荔湾区政府' not in label_1):
        not_liwan_total += 1
        if(label_1 == predict_1):
            not_liwan_correct += 1
print('label level 1 accurate: {}% ({}/{})'.format(true_lable_1 * 100 / total, true_lable_1, total))
print('tag level 2 accurate: {}% ({}/{})'.format(true_lable_2 * 100 / total, true_lable_2, total))
print('tag level 3 accurate: {}% ({}/{})'.format(true_lable_3 * 100 / total, true_lable_3, total))

print('{}% ({}/{})'.format(not_liwan_correct / not_liwan_total, not_liwan_correct, not_liwan_total))

ValueError: predict processes one line at a time (remove '\n')